In [ ]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from scipy.stats import t
from bisect import bisect
import scipy.io
import copy
from scipy.stats import norm
import numpy.matlib
%pylab inline
%matplotlib inline

In [ ]:
"""
This notebook implements the Logistic Regression in the 
"Cutting the Metropolis Hastings Budget" framework. 

The target distribution is the posterior for a logistic regression
model trained on MNIST dataset for classifying digits 7 vs 9. The 
dataset consisted of 12214 datapoints and we reduced the dimensionality
using PCA. 
"""

# get input data

mat = scipy.io.loadmat('minist7vs1.mat')
TrainLabel = mat['TrainLabel']
TrainLabel = np.reshape(TrainLabel, TrainLabel.shape[1],1)
TrainImg = mat['TrainImg']
# TestImg = mat['TestImg']
# TestLabel = mat['TestLabel']
print TrainLabel.shape
print TrainImg.shape
print type(TrainLabel)
print type(TrainImg)
print TrainImg[0].shape

In [ ]:
"""
Cutting the Metropolis Hastings Budget Method
"""
def random_walk_proposer(beta_t, sigmarw):
    """
    Input: beta_t  : previous parameters, 1-D array
           sigmarw : standard deviation, 1-D array
    """
    res = (np.random.multivariate_normal(beta_t, np.diag(sigmarw))).reshape((len(beta_t),))
    return res

def ll_data(beta_t, beta_n, X, Y):
    p_x_beta_t = 1/(1 + exp(-1* Y*np.dot(beta_t,X)))
    p_x_beta_n = 1/(1 + exp(-1* Y*np.dot(beta_n,X)))
    return log(p_x_beta_n) - log(p_x_beta_t)

def eval_U0(N):
    u = np.random.random()
    return 1/N*log(u)

def make_decision(l_list, N, U0):
    n = float(len(l_list))
    l_mean = sum(l_list)/n
    l_sq = ((np.array(l_list))**2).tolist()
    l_sq_mean = sum(l_sq)/n
    sl = sqrt((l_sq_mean - l_mean*l_mean)*n/(n-1))
    s = sl/sqrt(n) * sqrt(1 - (n-1)/(N-1))
    t_stats = abs((l_mean - U0)/s)
    delta = 1 - t.cdf(t_stats, n-1)
    return delta
    
def generate_parameter(beta_t, N, sigmarw, X, Y, batchsize, epsilon ):
    U0 = eval_U0(N)
    # generate new parameter
    P = len(X[0])
    sigma = (np.matlib.repmat(sigmarw, P,1)).reshape(P,)
    beta_n = random_walk_proposer(beta_t, sigma)
    
    # make decision
    l_list = []
    for i in range(batchsize):
        mid = ll_data(beta_t, beta_n, X[i], Y[i])
        l_list.append(mid)
    
    pos = batchsize
    delta = make_decision(l_list, N, U0)
    while (delta>epsilon) and (pos < N):
        mid = ll_data(beta_t, beta_n, X[pos], Y[pos])
        l_list.append(mid)
        pos = pos + 1
        delta = make_decision(l_list, N, U0)
    
    l_mean = sum(l_list)/float(len(l_list))
    if l_mean > U0:
        return (beta_n,1)
    else:
        return (beta_t,0)
    
def eval_cost(X,Y,beta):
    N = len(Y)
    cost = 0
    for i in range(N):
        sigmoid = 1/(1 + exp(-1* Y[i]*np.dot(beta,X[i])))
        if sigmoid < 0.5 : 
            cost = cost + 1
    return log(cost/float(N))
    
def train(X, Y, sigmarw, batchsize, epsilon, niter):
    N = len(Y) # number of total training data
    P = len(X[0]) # parameter dimension
    beta_t = np.ones(P)
    cost_list = []
    accept = []
    for i in range(niter):
        index = np.random.permutation(N)
        X = X[index]
        Y = Y[index]
        (beta_t, mid) = generate_parameter(beta_t, N, sigmarw, X,Y,batchsize,epsilon)
        cost = eval_cost(X,Y,beta_t)
        cost_list.append(cost)
    return cost_list

X = TrainImg
Y = TrainLabel
sigmarw = 0.01
batchsize = 50
epsilon = 0.02
niter = 2000
cost_list = train(X, Y, sigmarw, batchsize, epsilon, niter)

In [ ]:
plt.figure()
plt.plot(cost_list)
plt.show()
print len(cost_list)

In [ ]:
"""
The following part implements the logistic regression using our propsed Metropolis Hasitings Test Method

delta = log(f(\theta_t)p(\theta_p|theta_t)/f(\theta_p)/p(\theta_t|theta_p))

where f_

"""

